## Introducing more fields

This script uses the FieldMap and FieldMapping objects to add data from a table to a feature class.
This scripts shows the basic functionality:
- Creates a list of fields from the input and target datasets
- Creates a field map for each pair of fields from these two courses. This script assumes that the paired fields are at the same index position (this will be accounted for later)
- This script fromats a string to list the matching fields required for the match_fields parameter in the append tool

In [1]:
import arcpy

In [2]:
targetFC = r"C:\Demos\PYTS\SanDiego.gdb\CrimeTest"
inputTable = r"C:\Demos\PYTS\SanDiego.gdb\Crime_XY"

In [3]:
# Get a list of input fields
fieldNamesTarget = []
for field in arcpy.Describe(targetFC).fields:
    if field.type != "OID" and field.type != "Geometry": # Don't include the OID or Shape fields
        fieldNamesTarget.append(field)

fieldNamesInput = []
for field in arcpy.Describe(inputTable).fields:
    if "OID" not in field.name and field.type != "OID": # Don't include the OBJECTID or OID_ fields
        fieldNamesInput.append(field)

In [4]:
# Create the field map & field mappings objects
fieldMaps = []
fm0 = arcpy.FieldMap()
fm1 = arcpy.FieldMap()
fm2 = arcpy.FieldMap()
fms = arcpy.FieldMappings()

fm0.addInputField(targetFC, fieldNamesTarget[0].name)
fm0.addInputField(inputTable, fieldNamesInput[0].name)

fm1.addInputField(targetFC, fieldNamesTarget[1].name)
fm1.addInputField(inputTable, fieldNamesInput[1].name)

fm2.addInputField(targetFC, fieldNamesTarget[2].name)
fm2.addInputField(inputTable, fieldNamesInput[2].name)

fms.addFieldMap(fm0)
fms.addFieldMap(fm1)
fms.addFieldMap(fm2)

In [5]:
# Format a string: "XCoordinate X_WGS1984;YCoordinate Y_WGS1984;EventTimeDate start_date"
# This code here prints the paired fields
matchFields = ""
for fm in fms.fieldMappings:
    match = f"{fm.getInputFieldName(0)} {fm.getInputFieldName(1)};"
    matchFields += match
matchFields = matchFields[:-1]
print(matchFields)

EventTimeDate start_date;XCoordinate X_WGS1984;YCoordinate Y_WGS1984


In [6]:
arcpy.management.Append(
    inputs=inputTable,
    target=targetFC,
    schema_type="NO_TEST",
    field_mapping=fms,
    subtype="",
    expression="",
    match_fields=matchFields,
    update_geometry="NOT_UPDATE_GEOMETRY"
)

<Result 'C:\\Demos\\PYTS\\SanDiego.gdb\\CrimeTest'>

Opportunities for improvement are:
- Programaticaly create FieldMap objects for n desired field pairings
- Don't refer to fields based upon index positions in a list.